# North Dakota Weather Analysis

This is a report on the historical analysis of weather patterns in an area that approximately overlaps the area of the north part of North Dakota.

The data we will use here comes from [NOAA](https://www.ncdc.noaa.gov/). Specifically, it was downloaded from This [FTP site](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/).

We focused on six measurements:
* **TMIN, TMAX:** the daily minimum and maximum temperature.
* **TOBS:** The average temperature for each day.
* **PRCP:** Daily Percipitation (in mm)
* **SNOW:** Daily snowfall (in mm)
* **SNWD:** The depth of accumulated snow.

## Sanity-check: comparison with outside sources

<p>We start by comparing some of the general statistics with graphs that we obtained from a site called <a href="http://www.usclimatedata.com/climate/boston/massachusetts/united-states/usma0046" target="_blank">US Climate Data</a> The graph below shows the daily minimum and maximum temperatures for each month, as well as the total precipitation for each month.</p>

<p>&nbsp;</p>

<p><img alt="climate.png" src="data/climate.png" /></p>

<p>&nbsp;</p>

<p>We see that the min and max daily&nbsp;temperature agree with the ones we got from our data, once we translate Fahrenheit to Centigrade.</p>

<p>&nbsp;</p>

<p><img alt="TMINTMAX.png" src="data/TMINTMAX.png" style="height:300px; width:800px" /></p>

<p>To compare the precipitation&nbsp;we need to translate millimeter/day to inches/month. According to our analysis the average rainfall is 444 mm/year which translates to about 17.48 Inches&nbsp;per year. According to US-Climate-Data the average rainfall is closer to 17.82 inch per year. However, there is clear agreement that distribution of the average rainfall per month&nbsp;is corresponding to the distribution from US-Climate-Data.</p>

<p>&nbsp;<img alt="prcp.png" src="data/prcp.png" style="height:450px; width:600px" /></p>


## PCA analysis

For each of the six measurement, we compute the percentate of the variance explained as a function of the number of eigen-vectors used.

### Percentage of variance explained.
![VarExplained1.png](data/varexplained1.png)
The unusual plot for TMIN is due to the method we used to calculate covariance matrix. Since there are many NaN in the dataset, the covariance matrix may not be positive semi definite, which causes some eigenvalues being negative. However, this doesn't effect our analysis a lot. We could still continue on our analysis.

We see that the top 5 eigen-vectors explain 11% of variance for TMIN, 34% for TOBS and 20% for TMAX.

We conclude that of the three, TOBS is best explained by the top 5 eigenvectors. This is especially true for the first eigen-vector which, by itself, explains 23% of the variance.

![VarExplained2.png](data/varexplained2.png)

The top 5 eigenvectors explain 12% of the variance for PRCP and 9% for SNOW. Both are low values. On the other hand the top 5 eigenvectors explain 86% of the variance for SNWD. This means that these top 5 eigenvectors capture most of the variation in the snow signals. Based on that we will dig deeper into the PCA analysis for snow-depth.

It makes sense that SNWD would be less noisy than SNOW. That is because SNWD is a decaying integral of SNOW and, as such, varies less between days and between the same date on diffferent years.

## Analysis of snow depth

We choose to analyze the eigen-decomposition for snow-depth because the first 3 eigen-vectors explain 81% of the variance.

First, we graph the mean and the top 3 eigen-vectors.

We observe that the snow season is from November to the end of April, where the middle of February marks the peak of the snow-depth.
![snwd.png](data/snwd.png)

Next we interpret the eigen-functions. The first eigen-function (eig1) has a shape very similar to the mean function. The main difference is that the eigen-function is close to zero during october-december while the mean is not.  The interpretation of this shape is that eig1 represents the overall amount of snow above/below the mean, but without changing the distribution over time.

**eig2 and eig3** are similar in the following way. They all oscilate between positive and negative values. In other words, they correspond to changing the distribution of the snow depth over the winter months, but they don't change the total (much).

They can be interpreted as follows:
* **eig2:** less snow in nov - mid feb, more snow in mid feb - april.
* **eig3:** slightly more snow in jan - mid feb, slightly less snow in mid feb - april, less snow in nov - dec.


### Examples of reconstructions

#### Coeff1
Coeff1: most positive
![c1_positive.png](data/c1_positive.png)
Coeff1: most negative
![c1_negative.png](data/c1_negative.png)
Large positive values of coeff1 correspond to more than average snow. Low values correspond to less than average snow.

#### Coeff2
Coeff2: most positive
![c2_positive.png](data/c2_positive.png)
Coeff2: most negative
![c2_negative.png](data/c2_negative.png)

Large positive values of coeff2 correspond to a late snow season (most of the snowfall is after mid feb). Negative values for coeff2 correspond to an early snow season (most of the snow is before mid feb).
#### Coeff3
Coeff3: most positive
![c3_positive.png](data/c3_positive.png)
Coeff3: most negative
![c2_negative.png](data/c2_negative.png)

Large positive values of coeff2 correspond to a snow season with a single peak in january. Negative values of coeff3 correspond to a snow season with two spikes: one in the start of march, the other at the end of december



## The variation in the timing of snow is mostly due to year-to-year variation
In the previous section we see the variation of Coeff1, which corresponds to the total amount of snow, with respect to location. We now estimate the relative importance of location-to-location variation relative to year-by-year variation.

These are measured using the fraction by which the variance is reduced when we subtract from each station/year entry the average-per-year or the average-per-station respectively. Here are the results:

** coeff_1 **  
total RMS                   =  1404.59

RMS removing mean-by-station=  1227.22 , fraction explained =  12.6

RMS removing mean-by-year   =  861.92 , fraction explained =  38.6

** coeff_2 **  
total RMS                   =  646.12

RMS removing mean-by-station=  617.69 , fraction explained =  4.4

RMS removing mean-by-year   =  389.18 , fraction explained =  39.8

** coeff_3 **  
total RMS                   =  593.15

RMS removing mean-by-station=  571.39 , fraction explained =  3.7

RMS removing mean-by-year   =  304.36 , fraction explained =  48.7


We see that the variation by year explains more than the variation by station. However this effect is weaker consider coeff_1, which has to do with the total snowfall, vs. coeff_2,3 which, as we saw above have to do with the timining of snowfall. We see that for coeff_2,3 the stations explain 3-5% of the variance while the year explaines 40-50%.

## Analyzing Precipitation
From the PCA analysis in the previous section, we can find that precipitation is not well explained by the top 5 eigenvectors. To justify it, we extract all of the values of `res_3` which is the residual variance after the Mean and the first two eigenvectors have been subtracted out. Below is the cumulative distribution of `res_3`.
![cumulative.png](data/cumulative.png)
We can see that most instances have a large res_3 which means the residual variance after the Mean and the first two eigenvectors have been subtracted out is still very large. Therefore, we conclude that precipitation is not well explained by PCA analysis.

## Correlations between stations
It is likely to be hard to find correlations between the **amount** of rain on the same day in different stations. Because amounts of rain vary a lot between even close locations. It is more reasonable to try to compare whether or not it rained on the same day in different stations. As we see from the graph below, in our region it rains in about one fourth of the days.
![cdf_rainfall.png](data/cdf_rainfall.png)
We want to find a statistical test for rejecting the null hypothesis that says that the rainfall in the two locations is independent.

Using the inner product is too noisy, because you multiply the rainfall on the same day in two locations and that product can be very large - leading to a large variance and poor ability to discriminate.

An alternative is to ignore the amount of rain, and just ask whether it rained in both locations. We can then compute the probability associated with the number of overlaps under the null hypothesis.

Therefore, we calculate the normalized log probability for each pair of stations. The null hypothesis for this test is that the probability of it raining on any given day is independent for two stations. Below is the distribution of statistical significance.
![significance.png](data/significance.png)
As we can see from the graph, about 40% of pairs have significance $-frac{logP}{m}<0.05$, which means they are mainly independent. the remaining 60% are somehow dependent to others. Note that the spike of significance at 0.4 represents the p-value when comparing a station with itself. This is not important for data analysis.

To have a more intuitive impression, we plot the significance for those pairs.
![p_norm.png](data/p_norm.png)
The darker the pixel is, the more significant the datapoint is, which means the more dependent the two stations are. It's clear now that the first 70 stations are correlated and the last 50 stations are correlated, and stations between two groups are independent.